In [1]:
import pandas as pd

In [2]:
data2 = [
    {"product": 1, "location": "01", "demand": 100},
    {"product": 1, "location": "02", "demand": 110},
    {"product": 2, "location": "01", "demand": 120},
    {"product": 2, "location": "02", "demand": 90},
    {"product": 3, "location": "01", "demand": 70},
    {"product": 3, "location": "02", "demand": 80},
]

df2 = pd.DataFrame(data2)
df2.to_csv('df2.csv', index=False)
df2

,product,location,demand
0,1,01,100
1,1,02,110
2,2,01,120
3,2,02,90
4,3,01,70
5,3,02,80


In [3]:
# Датафрейм 3
data3 = [
    {"product": 1, "location": "01", "stock": 1000},
    {"product": 1, "location": "02", "stock": 400},
    {"product": 2, "location": "01", "stock": 300},
    {"product": 2, "location": "02", "stock": 250},
]

df3 = pd.DataFrame(data3)
df3.to_csv('df3.csv', index=False)
df3

,product,location,stock
0,1,01,1000
1,1,02,400
2,2,01,300
3,2,02,250


In [4]:
data = {'num_of_week': [1, 2, 3, 4, 5],
        'amount_days': [4, 7, 7, 7, 5],
        'start_date': ['01.06', '05.06', '12.06', '19.06', '26.06'],
        'end_date': ['04.06', '11.06', '18.06', '25.06', '30.06']}

dates_df = pd.DataFrame(data)
dates_df.to_csv('dates_df.csv', index=False)
dates_df

,num_of_week,amount_days,start_date,end_date
0,1,4,01.06,04.06
1,2,7,05.06,11.06
2,3,7,12.06,18.06
3,4,7,19.06,25.06
4,5,5,26.06,30.06


In [5]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType

In [6]:
spark = SparkSession.builder \
  .master("local[1]") \
  .appName("SessionLength") \
  .config("spark.executor.memory", "10g")\
  .config("spark.executor.cores", 5) \
  .config("spark.dynamicAllocation.enabled", "true") \
  .config("spark.dynamicAllocation.maxExecutors", 5) \
  .config("spark.shuffle.service.enabled", "true") \
  .getOrCreate()

In [7]:
# 1. Загрузить данные в PySpark.

demand_df = spark.read.format("csv").option("header", True).load("df2.csv")
demand_df.show()

stock_df = spark.read.format("csv").option("header", True).load("df3.csv")
stock_df.show()

dates_df = spark.read.format("csv").option("header", True).load("dates_df.csv")
dates_df.show()

+-------+--------+------+
|product|location|demand|
+-------+--------+------+
|      1|      01|   100|
|      1|      02|   110|
|      2|      01|   120|
|      2|      02|    90|
|      3|      01|    70|
|      3|      02|    80|
+-------+--------+------+

+-------+--------+-----+
|product|location|stock|
+-------+--------+-----+
|      1|      01| 1000|
|      1|      02|  400|
|      2|      01|  300|
|      2|      02|  250|
+-------+--------+-----+

+-----------+-----------+----------+--------+
|num_of_week|amount_days|start_date|end_date|
+-----------+-----------+----------+--------+
|          1|          4|     01.06|   04.06|
|          2|          7|     05.06|   11.06|
|          3|          7|     12.06|   18.06|
|          4|          7|     19.06|   25.06|
|          5|          5|     26.06|   30.06|
+-----------+-----------+----------+--------+



In [8]:
# Обработка данных
dates_df = dates_df.withColumn("start_date", col("start_date").cast("date"))

# Соединение demand_df и stock_df
combined_df = demand_df.join(stock_df, on=["product", "location"], how="outer")

# Добавление информации о датах
combined_df = combined_df.crossJoin(dates_df)

# Группировка данных и вычисление среднедневных остатков и количества проданных товаров
result_df = combined_df.groupBy("product", "location", "num_of_week", "amount_days").agg(
    expr("first(demand) as demand"),
    expr("first(stock) as stock"),
    (expr("first(stock) / 30") * col("amount_days")).alias("average_stock"),
).withColumn("forecast_sales", expr("demand * amount_days"))

# Округление значения average_stock до 2 знаков
result_df = result_df.withColumn("average_stock", round(col("average_stock"), 2))

# Отображение итоговой витрины данных
result_df.show(30)

+-------+--------+-----------+-----------+------+-----+-------------+--------------+
|product|location|num_of_week|amount_days|demand|stock|average_stock|forecast_sales|
+-------+--------+-----------+-----------+------+-----+-------------+--------------+
|      1|      01|          1|          4|   100| 1000|       133.33|         400.0|
|      1|      01|          2|          7|   100| 1000|       233.33|         700.0|
|      1|      01|          3|          7|   100| 1000|       233.33|         700.0|
|      1|      01|          4|          7|   100| 1000|       233.33|         700.0|
|      1|      01|          5|          5|   100| 1000|       166.67|         500.0|
|      1|      02|          1|          4|   110|  400|        53.33|         440.0|
|      1|      02|          2|          7|   110|  400|        93.33|         770.0|
|      1|      02|          3|          7|   110|  400|        93.33|         770.0|
|      1|      02|          4|          7|   110|  400|        93

In [9]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x0000021D11745D10>>